In [1]:
import pandas as pd
file = pd.read_csv('events_up_to_01062018.csv',low_memory=False)

In [2]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


In [6]:
def scrnres (x):
    c = x['screen_resolution'].value_counts().index
    if len(c)>0:
        return c[0]
    else:
        return 'Ninguno'
scrnres = file.groupby('person').apply(scrnres)
scrnres = pd.DataFrame(scrnres)
scrnres.columns = ['scrnres']

In [7]:
def opsis (x):
    c = x['operating_system_version'].value_counts().index
    if len(c)>0:
        return c[0]
    else:
        return 'Ninguno'
opsis = file.groupby('person').apply(opsis)
opsis = pd.DataFrame(opsis)
opsis.columns = ['opsis']

In [44]:
features_03 = pd.read_csv('features_03.csv')
features_03.head()

,person,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,vio_celulares,region,is_new
0,0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral,False,Unknown,True
1,00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct,True,Rio Grande do Sul,True
2,00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid,True,Minas Gerais,True
3,000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct,True,Sao Paulo,True
4,000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid,True,Minas Gerais,True


In [45]:
features_03.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38829 entries, 0 to 38828
Data columns (total 17 columns):
person                38829 non-null object
mayor_evento          38829 non-null object
prod_visto            38829 non-null object
dispos                38829 non-null object
compras               38829 non-null int64
visitas               38829 non-null int64
cant_prod_vistos      38829 non-null int64
primer_visita         38829 non-null object
ultima_visita         38829 non-null object
cant_eventos          38829 non-null int64
permanencia           38829 non-null int64
hace_cuanto_visito    38829 non-null int64
eventos_por_dia       38829 non-null float64
channel               38829 non-null object
vio_celulares         38829 non-null bool
region                38829 non-null object
is_new                38829 non-null bool
dtypes: bool(2), float64(1), int64(6), object(8)
memory usage: 4.5+ MB


In [94]:
opsis.head()

,scrnres
person,
0008ed71,Windows 10
00091926,Windows 7
00091a7a,Android 5.1.1
000ba417,Windows 10
000c79fe,Android 7


In [96]:
features_04 = features_04.set_index(features_04.columns[0]).join(opsis).join(scrnres)

In [46]:
f1 = features_03.groupby('prod_visto').agg({'person':'count'})

In [47]:
features_04 = features_03.merge(f1,how='left',on='prod_visto')

In [49]:
a = ['mayor_evento', 'dispos','channel','region']
to_merge = []
for c in a:
    b = features_04.groupby(c).agg({features_04.columns[0]:'count'})
    features_04 = features_04.merge(b,how='left',on=c)

In [50]:
features_04.head()

,person_x_x_x_x_x,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,...,eventos_por_dia,channel,vio_celulares,region,is_new,person_y,person_x_y,person_x_x_y,person_x_x_x_y,person_x_x_x_x_y
0,0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,...,6.0,Referral,False,Unknown,True,1699,717,17408,3169,6612
1,00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,...,16.0,Direct,True,Rio Grande do Sul,True,2889,30032,17408,5438,1122
2,00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,...,10.0,Paid,True,Minas Gerais,True,2889,1521,20152,24190,3306
3,000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,...,20.6,Direct,True,Sao Paulo,True,74,30032,17408,5438,11024
4,000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,...,17.0,Paid,True,Minas Gerais,True,1404,1372,20152,24190,3306


In [69]:
def channel_inicial (x):
    vsc = x.loc[x.event=='visited site']['channel']
    if vsc.size>0:
        return vsc.values[0]
    else:
        return 'Unknown'
ci = file.groupby('person').apply(channel_inicial)

In [70]:
ci.value_counts()

Paid        24001
Organic      7452
Direct       4452
Referral     1757
Unknown       588
Social        515
Email          64
dtype: int64

In [72]:
features_04 = features_04.join(pd.DataFrame(ci))

In [73]:
features_04.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,channel,vio_celulares,region,is_new,person_y,person_x_y,person_x_x_y,person_x_x_x_y,person_x_x_x_x_y,0
person_x_x_x_x_x,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,...,Referral,False,Unknown,True,1699,717,17408,3169,6612,Referral
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,...,Direct,True,Rio Grande do Sul,True,2889,30032,17408,5438,1122,Direct
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,...,Paid,True,Minas Gerais,True,2889,1521,20152,24190,3306,Paid
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,...,Direct,True,Sao Paulo,True,74,30032,17408,5438,11024,Direct
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,...,Paid,True,Minas Gerais,True,1404,1372,20152,24190,3306,Paid


In [77]:
file.event.value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [79]:
def notificaciones (x):
    return x.loc[x.event=='lead'].index.size
notif = file.groupby('person').apply(notificaciones)
notif = pd.DataFrame(notif)
notif.columns = ['lead']

In [82]:
features_04 = features_04.join(notif)

In [102]:
features_04.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,is_new,person_y,person_x_y,person_x_x_y,person_x_x_x_y,person_x_x_x_x_y,0notificaciones,0,opsis,scrnres
person_x_x_x_x_x,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,...,True,1699,717,17408,3169,6612,Referral,0,Windows 10,1920x1080
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,...,True,2889,30032,17408,5438,1122,Direct,0,Windows 7,1024x768
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,...,True,2889,1521,20152,24190,3306,Paid,0,Android 5.1.1,360x640
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,...,True,74,30032,17408,5438,11024,Direct,0,Windows 10,1024x768
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,...,True,1404,1372,20152,24190,3306,Paid,0,Android 7,360x640


In [103]:
a = ['opsis','scrnres']
to_merge = []
for c in a:
    b = features_04.groupby(c).agg({features_04.columns[0]:'count'})
    features_04 = features_04.merge(b,how='left',on=c)

In [104]:
features_04.head()

,mayor_evento_x_x,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,person_x_y,person_x_x_y,person_x_x_x_y,person_x_x_x_x_y,0notificaciones,0,opsis,scrnres,mayor_evento_y,mayor_evento_x_y
0,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,...,717,17408,3169,6612,Referral,0,Windows 10,1920x1080,6565,1355
1,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,...,30032,17408,5438,1122,Direct,0,Windows 7,1024x768,8660,1141
2,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,...,1521,20152,24190,3306,Paid,0,Android 5.1.1,360x640,1870,12896
3,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,...,30032,17408,5438,11024,Direct,0,Windows 10,1024x768,6565,1141
4,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,...,1372,20152,24190,3306,Paid,0,Android 7,360x640,4266,12896
